# Necessary import


In [1]:
#pip install spacy

In [73]:
import itertools
import os
import re
import secrets
import string
import numpy as np
import pandas as pd
import spacy
import json
import nltk

import spacy
import tomotopy as tp
from nltk.corpus import stopwords

from itertools import chain

from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec, Phrases, KeyedVectors
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess

import gensim.corpora as corpora
from gensim.models import CoherenceModel

from nltk.corpus import wordnet
# from pattern.en import pluralize, singularize
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

from spacy.parts_of_speech import IDS as POS_map

# Used functions

In [3]:
def dummy_word_split(texts):
    """Function identifying words in a sentence in a really dummy way.
        
        Argument:
            - texts (list of str): a list of raw texts in which we'd like to identify words
            
        Return:
            - list of list containing each word separately.
    """
    texts_out = []
    for text in texts:
        texts_out.append(text.split(" "))
        
    return texts_out

In [4]:
def compute_word_occurences(texts):
    """You have to define this function yourself. """
    
    words = itertools.chain.from_iterable(texts)
    
    word_count = pd.Series(words).value_counts()
    word_count = pd.DataFrame({"Word": word_count.index, "Count": word_count.values})

    return word_count

### Ensuring data quality

In [5]:
def check_data_quality(texts):
    """Check wheter all the dataset is conform to the expected behaviour."""
    assert all([isinstance(t, str) for t in texts]), "Input data contains something different than strings."
    assert all([t != np.nan for t in texts]), "Input data contains NaN values."
    
    return True

In [6]:
def force_format(texts):
    return [str(t) for t in texts]

### Filtering texts

In [7]:
def filter_text(texts_in):
    """Removes incorrect patterns from a list of texts, such as hyperlinks, bullet points and so on"""
    
    texts_out = re.sub(r'https?:\/\/[A-Za-z0-9_.-~\-]*', ' ', texts_in, flags=re.MULTILINE)
    texts_out = re.sub(r'[(){}\[\]<>]', ' ', texts_out, flags=re.MULTILINE)
    texts_out = re.sub(r'&amp;#.*;', ' ', texts_out, flags=re.MULTILINE)
    texts_out = re.sub(r'&gt;', ' ', texts_out, flags=re.MULTILINE)
    texts_out = re.sub(r'â€™', "'", texts_out, flags=re.MULTILINE)
    texts_out = re.sub(r'\s+', ' ', texts_out, flags=re.MULTILINE)
    texts_out = re.sub(r'&#x200B;', ' ', texts_out, flags=re.MULTILINE)
    texts_out = re.sub(r'\-', ' ', texts_out, flags=re.MULTILINE)
    
    # Mail regex
    # This regex is correct but WAY TOO LONG to process. So we skip it with a simpler version
    # texts_out = re.sub(r"(?i)(?:[a-z0-9!#$%&'*+\/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+\/=?^_`{|}~-]+)*|\"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*\")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])", '', texts_out, flags=re.MULTILINE)
    texts_out = re.sub(r'[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+\.[a-zA-Z0-9-_.]+', '', texts_out, flags=re.MULTILINE)
    # Phone regex
    # This regex is correct but WAY TOO LONG to process. So we skip it with a simpler version
    # texts_out = re.sub(r".*?(\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}).*?", '', texts_out, flags=re.MULTILINE)
    texts_out = re.sub(r"\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}", '', texts_out, flags=re.MULTILINE)
    # Remove names in twitter
    texts_out = re.sub(r'@\S+( |\n)', '', texts_out, flags=re.MULTILINE)

    # Remove starts commonly used on social media
    texts_out = re.sub(r'\*', '', texts_out, flags=re.MULTILINE)
    return texts_out

### Unifying texts & converting sentences to list of words

In [8]:
def sent_to_words(sentences):
    """Converts sentences to words.

    Convert sentences in lists of words while removing the accents and the punctuation.

    @param:
        sentences: a list of strings, the sentences we want to convert
    @return
        A list of words' lists.
    """
    for sentence in tqdm(sentences):
        yield (simple_preprocess(str(sentence), deacc=True))

### Removing useless words

In [9]:
#import nltk
#nltk.download('stopwords')

In [10]:
def get_stopwords(additional_stopwords=[]):
    """Return a list of english stopwords, that can be augmented by using a stopwords file or a list of stopwords

    Args:
        filepath (str, optional): path to a text file where each line is a stopword
        additional_stopwords (list of str, optional): list of string representing stopwords
    Returns:
        List of strings representing stopwords
    """
    # Loading standard english stop words
    with open('stopwords.txt', 'r') as f:
        stop_w = f.readlines()
    stopwords = [s.rstrip() for s in stop_w]

    # Adding stop words from sklearn
    stopwords = list(text.ENGLISH_STOP_WORDS.union(stopwords))

    # Adding words from a list if specified
    if additional_stopwords:
        stopwords += additional_stopwords

    # Removing duplicates
    stopwords = list(set(stopwords))

    # Removing some \n that were included in the native stopwords of sklearn ... WHY?
    stopwords = [s.replace("\n", "") for s in stopwords]

    stopwords = sorted(stopwords, key=str.lower)

    return stopwords


### Creating n-grams

In [11]:
def create_bigrams(texts, bigram_count=15, threshold=10, convert_sent_to_words=False, as_str=True):
    """Identify bigrams in texts and return the texts with bigrams integrated"""
    if convert_sent_to_words:
        texts = list(sent_to_words(texts))
    
    bigram_model = Phraser(Phrases(texts, min_count=bigram_count, threshold=threshold))
    
    if as_str:
        return [" ".join(bigram_model[t]) for t in texts]

    else:
        return [bigram_model[t] for t in texts]

def create_trigrams(texts, trigram_count=15, threshold=10, convert_sent_to_words=False, as_str=True):
    """Identify trigrams in texts and return the texts with trigrams integrated"""
    if convert_sent_to_words:
        texts = list(sent_to_words(texts))
    
    bigram_model = Phraser(Phrases(texts, min_count=bigram_count, threshold=threshold))
    
    if as_str:
        return [" ".join(bigram_model[t]) for t in texts]

    else:
        return [bigram_model[t] for t in texts]


### Stemming / Lemmatization & Part-of-Speech filtering

In [12]:
#conda install -c conda-forge spacy-model-en_core_web_md

In [13]:
def lemmatize_texts(texts, 
                    allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'], 
                    forbidden_postags=[], 
                    as_sentence=False, 
                    get_postags=False, 
                    spacy_model=None):
    """Lemmatize a list of texts.
    
            Please refer to https://spacy.io/api/annotation for details on the allowed
        POS tags.
        @params:
            - texts_in: a list of texts, where each texts is a string
            - allowed_postags: a list of part of speech tags, in the spacy fashion
            - as_sentence: a boolean indicating whether the output should be a list of sentences instead of a list of word lists
        @return:
            - A list of texts where each entry is a list of words list or a list of sentences
        """
    texts_out = []
    
    if allowed_postags and forbidden_postags:
        raise ValueError("Can't specify both allowed and forbidden postags")

    if forbidden_postags:
        allowed_postags = list(set(POS_map.keys()).difference(set(forbidden_postags)))

    if not spacy_model:
        print("Loading spacy model")
        spacy_model = spacy.load('en_core_web_md')

    print("Beginning lemmatization process")
    total_steps = len(texts)

    docs = spacy_model.pipe(texts)

    for i, doc in tqdm(enumerate(docs), total=total_steps):
        if get_postags:
            texts_out.append(["_".join([token.lemma_, token.pos_]) for token in doc if token.pos_ in allowed_postags])
        else:
            texts_out.append(
                [token.lemma_ for token in doc if token.pos_ in allowed_postags])
    
    if as_sentence:
        texts_out = [" ".join(text) for text in texts_out]
        
    return texts_out


### Using pre-trained Word2Vec representations from spacy

In [14]:
#conda install -c conda-forge spacy-model-en_core_web_sm

In [15]:

nlp = spacy.load('en_core_web_sm')
nlp("this is a course")[3].vector

array([ 0.535709  ,  0.0370416 ,  0.7242172 ,  0.49471536,  0.80562615,
        0.94397926, -0.32036173,  0.1282663 ,  0.48324287, -0.7638728 ,
        0.01772588, -0.85815585, -0.01381579, -0.19521122,  1.1966841 ,
       -0.29897153,  0.49273318,  0.98047674,  0.05885285,  0.47619057,
        0.5893681 , -0.03815475, -0.0924111 ,  1.3388768 ,  0.45786104,
       -0.7812397 , -1.115696  ,  0.5145061 , -0.10024548, -0.15883371,
        1.0248507 , -1.4965878 ,  0.81161046, -0.2256628 ,  0.82724553,
       -0.10684998, -0.05577912, -1.5601915 , -0.18874454,  0.3366524 ,
       -1.7478721 , -1.0365617 , -0.13422352,  0.7865871 ,  1.0538278 ,
       -0.90179   ,  0.6694306 , -0.32928434, -0.28328714,  0.43673193,
        0.27765962,  0.13045381, -0.587474  , -0.68984103, -0.10532296,
       -0.82711416, -0.15132898, -0.1912263 ,  1.2880498 , -0.7229643 ,
        0.7820295 , -0.89512396, -0.00276873,  0.23332623,  0.57228655,
       -0.02196151,  0.6592536 , -1.1398258 , -1.5783123 , -0.41

In [16]:
def get_word_embeddings(texts, occurences):
    words = []
    vector_representations = []

    for word in occurences["Word"].head(100):
        words.append(word)
        vector_representations.append(nlp(word)[0].vector)

    return pd.DataFrame({"Words": words, "Vector": vector_representations})

In [17]:
nlp = spacy.load('en_core_web_sm')

# Loads the data

In [18]:
dataset = pd.read_json("../../data/News_Category_Dataset_v2.json", lines=True, dtype={"headline": str})

# Analyser le corpus de texte pour en extraire ses caractéristiques spécifiques (taille moyenne, types de mots utilisés, mots les plus fréquents, stopwords, etc.)

1. **Ensuring data quality.** You have to make sure that there's no N/A in your data and that everything is in the good format shape. Having this as the entrance of your pipeline will save you a lot of time in the long run, so try defining it thoroughly.


2. **Filtering texts from unwanted characters**. Especially if you get data from web, you'll end up with HTML tags or encoding stuff that you don't need in your texts. Before applying anything to them, you need to get them cleaned up. Here, try removing the dates and the punctuation for instance.


3. **Unify your texts**. (*This is topic modeling specific*). You don't want to make the difference between a word at the beginning of a phrase of in the middle of it here. You should unify all your words by lowercasing them and deaccenting them as well.


4. **Converting sentences to lists of words**. Some words aren't needed for our analyses, such as *your*, *my*, etc. In order to remove them easily, you have to convert your sentences to lists of words. You can use the dummy function defined above but I'd advised against it. Try finding a function that does that smoothly in [gensim.utils](https://radimrehurek.com/gensim/utils.html)!


5. **Remove useless words**. You need to remove useless words from your corpus. You have two approaches: [use a hard defined list of stopwords](https://www.analyticsvidhya.com/blog/2019/08/how-to-remove-stopwords-text-normalization-nltk-spacy-gensim-python/) or rely on TF-IDF to identify useless words. The first is the simplest, the second might yield better results!


6. **Creating n-grams**. If you look at New York, it is composed of two words. As a result, a word count wouldn't really return a true count for *New York* per se. In NLP, we represent New York as New_York, which is considered a single word. The n-gram creation consists in identifying words that occur together often and regrouping them. It boosts interpretability for topic modeling in this case.


7. **Stemming / Lemmatization**. Shouldn't run, running, runnable be grouped and counted as a single word when we're identifying discussion topics? Yes, they should. Stemming is the process of cutting words to their word root (run- for instance) quite brutally while lemmatization will do the same by identifying the kind of word it is working on. You should convert the corpus words into those truncated representations to have a more realistic word count.


8. **Part of speech tagging**. POS helps in the identification of verbs, nouns, adjectives, etc. For topic models, it is a good idea to work only on verbs and nouns. Adjectives don't convey info about the actual underlying topic discussed at hand.

## Ensuring the data quality

In [19]:
dataset

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [20]:
source = force_format(dataset["headline"])
print(f"Is the dataset passing our data quality check?\n{check_data_quality(source)}")

Is the dataset passing our data quality check?
True


In [21]:
source[:10]


['There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV',
 "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song",
 'Hugh Grant Marries For The First Time At Age 57',
 "Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork",
 'Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog',
 "Morgan Freeman 'Devastated' That Sexual Harassment Claims Could Undermine Legacy",
 "Donald Trump Is Lovin' New McDonald's Jingle In 'Tonight Show' Bit",
 'What To Watch On Amazon Prime That’s New This Week',
 "Mike Myers Reveals He'd 'Like To' Do A Fourth Austin Powers Film",
 'What To Watch On Hulu That’s New This Week']

## Filtering texts from unwanted characters

In [22]:
texts = [filter_text(t) for t in source]

In [23]:
texts[:10]

['There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV',
 "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song",
 'Hugh Grant Marries For The First Time At Age 57',
 "Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork",
 'Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog',
 "Morgan Freeman 'Devastated' That Sexual Harassment Claims Could Undermine Legacy",
 "Donald Trump Is Lovin' New McDonald's Jingle In 'Tonight Show' Bit",
 'What To Watch On Amazon Prime That’s New This Week',
 "Mike Myers Reveals He'd 'Like To' Do A Fourth Austin Powers Film",
 'What To Watch On Hulu That’s New This Week']

## Unify your texts

In [24]:
texts = list(sent_to_words(texts))

100%|██████████| 200853/200853 [00:03<00:00, 53718.39it/s]


In [25]:
texts[:3]

[['there',
  'were',
  'mass',
  'shootings',
  'in',
  'texas',
  'last',
  'week',
  'but',
  'only',
  'on',
  'tv'],
 ['will',
  'smith',
  'joins',
  'diplo',
  'and',
  'nicky',
  'jam',
  'for',
  'the',
  'world',
  'cup',
  'official',
  'song'],
 ['hugh', 'grant', 'marries', 'for', 'the', 'first', 'time', 'at', 'age']]

## Converting sentences to lists of words

In [26]:
#stopwords = get_stopwords(additional_stopwords=["trump"])

## Remove useless words

In [27]:
nltk.download('stopwords', quiet=True)

st_words = stopwords.words('english')
st_words.extend(['trump', ''])

In [28]:
texts = [[word for word in txt if word not in st_words] for txt in tqdm(texts)]

100%|██████████| 200853/200853 [00:03<00:00, 65552.59it/s]


In [29]:
texts[:3]

[['mass', 'shootings', 'texas', 'last', 'week', 'tv'],
 ['smith',
  'joins',
  'diplo',
  'nicky',
  'jam',
  'world',
  'cup',
  'official',
  'song'],
 ['hugh', 'grant', 'marries', 'first', 'time', 'age']]

## Creating n-grams

In [30]:
texts = create_bigrams(texts)

In [31]:
texts[:3]

['mass_shootings texas last week tv',
 'smith joins diplo nicky jam world_cup official song',
 'hugh grant marries first_time age']

## Stemming / Lemmatization

In [32]:
l_texts = lemmatize_texts(texts,
                allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'X', 'PROPN'], 
                #allowed_postags=['ADJ', 'VERB', 'ADV', 'X', 'PROPN'],
                get_postags=False)

Loading spacy model
Beginning lemmatization process


100%|██████████| 200853/200853 [02:29<00:00, 1340.39it/s]


In [33]:
l_texts[:3]

[['mass_shooting', 'texas', 'last', 'week', 'tv'],
 ['smith', 'join', 'diplo', 'nicky', 'jam', 'world_cup', 'official', 'song'],
 ['hugh', 'grant', 'marrie', 'first_time', 'age']]

In [34]:
occurences = compute_word_occurences(l_texts)

In [35]:
occurences.head()

,Word,Count
0,photo,11840
1,new,6024
2,make,5789
3,video,5729
4,say,5272


In [36]:
df = get_word_embeddings(texts, occurences)

In [37]:
df

,Words,Vector
0,photo,"[-1.2354703, -0.34269822, 0.6657233, 0.9129891..."
1,new,"[-1.1592835, 0.4829545, 0.78554547, 0.34790865..."
2,make,"[-0.15756671, -0.34884667, -0.02117005, 1.0657..."
3,video,"[-0.83165646, -0.63011926, 0.70887375, 1.24560..."
4,say,"[0.26212564, -1.3126553, 0.09304688, 0.2069553..."
...,...,...
95,break,"[0.061404377, -0.24197388, 0.11074421, -0.2865..."
96,reveal,"[-0.28347433, 0.5458174, 0.14895129, 0.8303557..."
97,style,"[-0.52506196, 0.31397888, 0.37584454, 0.782197..."
98,turn,"[-0.61745894, -0.27704814, 0.49550354, 0.74584..."


In [40]:
sa_l_texts = l_texts[:100000]

## HDP Training


In [41]:
term_weight = tp.TermWeight.IDF
#term_weight = tp.TermWeight.ONE

# alpha : concentration coeficient of Dirichlet Process for document-table
# gamma : concentration coeficient of Dirichlet Process for table-topic

hdp = tp.HDPModel(tw=term_weight, min_cf=5, rm_top=3, gamma=1, alpha=0.1, initial_k=5, seed=99999)

#for vec in tqdm(l_texts):
for vec in sa_l_texts:
    if vec: 
        hdp.add_doc(vec)
        
        
# Initiate sampling burn-in  (i.e. discard 100 first iterations)
hdp.burn_in = 100
hdp.train(0)
print('Num docs:', len(hdp.docs), ', Vocab size:', hdp.num_vocabs,
      ', Num words:', hdp.num_words)
print('Removed top words:', hdp.removed_top_words)    

Num docs: 99957 , Vocab size: 12137 , Num words: 561282
Removed top words: ['donald', 'say', 'new']


In [42]:
# Train model
for i in tqdm(range(0, 1000, 100)):
    hdp.train(100) # 100 iterations at a time
    print('Iteration: {}\tLog-likelihood: {}\tNum. of topics: {}'.format(i, hdp.ll_per_word, hdp.live_k))

 10%|█         | 1/10 [00:12<01:56, 12.97s/it]

Iteration: 0	Log-likelihood: -8.015742403879353	Num. of topics: 360


 30%|███       | 3/10 [01:36<04:07, 35.30s/it]

Iteration: 100	Log-likelihood: -7.802733823049065	Num. of topics: 963
Iteration: 200	Log-likelihood: -7.734014945753611	Num. of topics: 1110


 50%|█████     | 5/10 [03:04<03:24, 40.85s/it]

...

Iteration: 500	Log-likelihood: -7.660019964634503	Num. of topics: 1264
Iteration: 600	Log-likelihood: -7.646072234693017	Num. of topics: 1300
Iteration: 700	Log-likelihood: -7.628643926781195	Num. of topics: 1345


 90%|█████████ | 9/10 [06:11<00:45, 45.60s/it]

Iteration: 800	Log-likelihood: -7.620407921558563	Num. of topics: 1347


100%|██████████| 10/10 [07:03<00:00, 42.36s/it]

Iteration: 900	Log-likelihood: -7.6123434574443	Num. of topics: 1379


In [43]:
def get_hdp_topics(hdp, top_n):
    '''Wrapper function to extract topics from trained tomotopy HDP model 
    
    ** Inputs **
    hdp:obj -> HDPModel trained model
    top_n: int -> top n words in topic based on frequencies
    
    ** Returns **
    topics: dict -> per topic, an arrays with top words and associated frequencies 
    '''
    
    # Get most important topics by # of times they were assigned (i.e. counts)
    sorted_topics = [k for k, v in sorted(enumerate(hdp.get_count_by_topics()), key=lambda x:x[1], reverse=True)]

    topics=dict()
    
    # For topics found, extract only those that are still assigned
    for k in sorted_topics:
        if not hdp.is_live_topic(k): continue # remove un-assigned topics at the end (i.e. not alive)
        topic_wp =[]
        for word, prob in hdp.get_topic_words(k, top_n=top_n):
            topic_wp.append((word, prob))

        topics[k] = topic_wp # store topic word/frequency array
        
    return topics

In [44]:
topics = get_hdp_topics(hdp, top_n=5) # changing top_n changes no. of words displayed

In [45]:
k = 543
print("source : " + str(source[k]))

source : Angry White Dude's Rant About People Speaking Spanish In NYC Goes Viral


In [46]:
len(topics)

1379

In [47]:
topics[0]

[('gop', 0.003511444665491581),
 ('obama', 0.0033199803438037634),
 ('republican', 0.003071588696911931),
 ('hillary_clinton', 0.0027717584744095802),
 ('call', 0.0026542795822024345)]

In [48]:
#hdp.save('sample_hdp_model.bin')
# Iteration: 900	Log-likelihood: -7.124920004899671	Num. of topics: 2002

In [49]:
#hdp = tp.HDPModel.load('sample_hdp_model.bin')

### Cohérence et Inférence


In [50]:
# Build gensim objects


vocab = corpora.Dictionary(sa_l_texts)
corpus = [vocab.doc2bow(words) for words in sa_l_texts]

# Build topic list from dictionary
topic_list=[]
for k, tups in topics.items():
    topic_tokens=[]
    for w, p in tups:
        topic_tokens.append(w)

    topic_list.append(topic_tokens)

In [53]:
cm = CoherenceModel(topics=topic_list, corpus=corpus, dictionary=vocab,
                    texts=sa_l_texts, coherence='c_v')
    
cm.get_coherence()

0.666191127034379

### Résultats Matrice de Cohérence
En test sur une échantillon de 5000 headlind, nous avons pu monter à un taut de 0.66 env.
Cependant, en vérifiant certains headline avec le topic associé manuellement, on se rend compte que les résultats n'étaient pas sattisfaisant.

En utilisant la totalité des headlines (200k) nous avons un taut à 0.55. En vérifiant manuellement les résultats sont plus logiques.

Apres d'autres essaie avec 100k headlines, nous avons un taut de 0.66 et les topics nous semblent plus logique que les résultats précédents.

Ont aurraient put continuer à faire du fine tunning. Mais l'entrainnement avec 100k prennais 10 min/entrainnement.

In [61]:
#sarcasme ligne = 6208
ligne = 6208
#ligne = 65923


test_doc = sa_l_texts[ligne]
doc_inst = hdp.make_doc(test_doc)
topic_dist, ll = hdp.infer(doc_inst)
topic_idx = np.array(topic_dist).argmax()

print("Id du topic ligne {} :\t {}".format(ligne, topic_idx))
print("\nHeadline : " + source[ligne])
print("Lematisation : " + str(sa_l_texts[ligne]))

Id du topic ligne 6208 :	 1

Headline : Kevin Hart Drops An F-Bomb In Awkward NFL Network Interview
Lematisation : ['kevin_hart', 'drop', 'bomb', 'awkward', 'nfl', 'network', 'interview']


In [62]:
hdp.get_topic_words(topic_idx)

[('star', 0.004240021575242281),
 ('make', 0.0035927616991102695),
 ('get', 0.003574989503249526),
 ('show', 0.0034358229022473097),
 ('watch', 0.003092397004365921),
 ('first', 0.002923456719145179),
 ('good', 0.00280915480107069),
 ('taylor_swift', 0.0027206295635551214),
 ('fan', 0.002608110662549734),
 ('movie', 0.0025976519100368023)]

In [63]:
print(dataset.loc[ligne,["category","headline","short_description"]])
print('\n' + dataset["link"][ligne])

category                                                        COMEDY
headline             Kevin Hart Drops An F-Bomb In Awkward NFL Netw...
short_description                                "I've been drinking."
Name: 6208, dtype: object

https://www.huffingtonpost.com/entry/kevin-hart-nfl-f-bomb_us_5a7816f2e4b06ee97af49a0f


### Comparaison

In [67]:
pip install pyLDAvis

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=2354d1b342993e11283f1cff22e4e95df22e632ea92d36bb4fb20284cfe12098
  Stored in directory: c:\users\dana\appdata\local\pip\cache\wheels\e4\7b\98\b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [80]:
hdp.summary()

<Basic Info>
| HDPModel (current version: 0.12.2)
| 99957 docs, 561282 words
| Total Vocabs: 31959, Used Vocabs: 12137
| Entropy of words: 8.33824
| Entropy of term-weighted words: 8.64782
| Removed Vocabs: donald say new
|
<Training Info>
| Iterations: 1000, Burn-in steps: 100
| Optimization Interval: 10
| Log-likelihood per word: -7.61234
|
<Initial Parameters>
| tw: TermWeight.IDF
| min_cf: 5 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 3 (the number of top words to be removed)
| initial_k: 5 (the initial number of topics between 2 ~ 32767 The number of topics will be adjusted for data during training)
| alpha: 0.1 (concentration coeficient of Dirichlet Process for document-table )
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| gamma: 1.0 (concentration coeficient of Dirichlet Process for table-topic)
| seed: 99999 (random seed)
| trained in version 0.12.2
|
<Parameters>
| alpha (concentration coeficient

In [98]:
# en cherchant à debud l'affichage via pyLDAvis
# https://github.com/bab2min/tomotopy/issues/74
#Avec tomotopy, il faut convertir 

ldamodel, _ = hdp.convert_to_lda(topic_threshold=0.001)

# pyLDAvis에 입력하여 시각화하기
topic_term_dists = np.stack([ldamodel.get_topic_word_dist(k) for k in range(ldamodel.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in ldamodel.docs])
doc_lengths = np.array([len(doc.words) for doc in ldamodel.docs])
vocab = list(ldamodel.used_vocabs)
term_frequency = ldamodel.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency
)
pyLDAvis.save_html(prepared_data, 'ldavisModelHDP.html')

X:\Program\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
